In [2]:
import numpy as np
import networkx as nx
import scipy.sparse as sp
import pandas as pd
from itertools import combinations, chain
import shelve
from getpass import getuser
from data.load_functions import *

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [22]:
newdict = {1:0, 2:0, 3:0, 4:1}
newdict.keys()
u_value = set(val for val in newdict.values())
len(set( val for val in newdict.values()))

2

In [27]:
# Loading Gene data (PPI)
ppi, gene2idx = load_ppi(fname='data/clean_data/ppi_mini.csv')
ppi_adj = nx.adjacency_matrix(ppi)
ppi_degrees = np.array(ppi_adj.sum(axis=0)).squeeze() 
ppi_genes = ppi.number_of_nodes() # Number of genes (nodes)
# Loading individual side effects
stitch2se, semono2name, semono2idx = load_mono_se(fname='data/clean_data/mono_mini.csv')
n_semono = len(semono2name)
print('Number of individual side effects: ', n_semono)
# Loading Target data (DTI)
stitch2proteins = load_targets(fname='data/clean_data/targets_mini.csv')
dti_drugs = len(pd.unique(list(stitch2proteins.keys())))
dti_genes = len([val for val in stitch2proteins.values()])
print('Number of genes in DTI:', dti_genes)
print('Number of drugs in DTI:', dti_drugs)
# Loading Drug data (DDI)
combo2stitch, combo2se, secombo2name, drug2idx = load_combo_se(fname='data/clean_data/combo_mini.csv')
n_secombo = len(secombo2name)
# Loading Side effect data (features)
stitch2se, semono2name, semono2idx = load_mono_se(fname='data/clean_data/mono_mini.csv')
# Loading protein features
PF = pd.read_csv('data/clean_data/genes_mini.csv', sep=',',header=None).to_numpy()
ddi_drugs = len(drug2idx)
print('Number of drugs: ', ddi_drugs)

Number of ppi interactions: 308950
Number of genes: 16261
Number of individual side effects:  9928
Number fo DTI interactions: 16367
Number of genes in DTI: 235
Number of drugs in DTI: 235
Drug combinations: 15229 Side effects: 3
Drug-drug interactions: 16030
Number of drugs:  528


## Object saving
`numpy.save()` only to save arrays. One per file

## Formatted string saving
```
f=open('file.txt', 'rb')
f.write(...)
```

In [ ]:
scores = np.load('results/acc_test.npy',allow_pickle=True)

In [ ]:
mem = np.load('results/memtime_test.npy',allow_pickle=True)
print('CMM values')
print('Virtual memory: %.3fGb'%(mem[0]*1e-9))
print('RSS memory: %.3fGb' % (mem[1]*1e-9))
print('Total time: ', str(datetime.timedelta(seconds=round(mem[2]))))

In [ ]:
plot(scores[:,1])

## Use shelve to save and load

In [ ]:
my_shelf = shelve.open('data.out','n')

In [ ]:
my_shelf['mem'] = mem
my_shelf['scores'] = scores

In [ ]:
my_shelf['mem']

In [ ]:
my_shelf.close()

In [ ]:
del mem
del scores

In [ ]:
print(scores)

In [7]:
my_shelf = shelve.open('session_juan_NSE_3.out')


In [8]:
my_shelf.keys()

['ppi_adj', 'dti_adj', 'ddi_adj_list']

In [9]:
for key in my_shelf:
    globals()[key]=my_shelf[key]

In [10]:
my_shelf.close()

In [11]:
ppi_adj

<16261x16261 sparse matrix of type '<type 'numpy.int64'>'
	with 617900 stored elements in Compressed Sparse Row format>

## Load adjacency matrices

In [ ]:
adj_mats_orig = np.load('adjs.npy',allow_pickle=True)

In [ ]:
adj_mats_orig[0].todense()